<a href="https://colab.research.google.com/github/panghanwu/machine_learning_Elwing/blob/main/embedding_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

語言是人類在有意及無意之間訂定出來溝通方式，衍伸出抽象的概念及情緒含意，而難以有系統性的歸類。使用深度學習訓練出詞向量（word vector）模型是一種有效的逼近方法。

詞向量維度 128~512

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 1s 0us/step


In [2]:
import glob

glob.glob('/root/.keras/datasets/aclImdb/train/pos/*')[0:6]

['/root/.keras/datasets/aclImdb/train/pos/987_8.txt',
 '/root/.keras/datasets/aclImdb/train/pos/8053_8.txt',
 '/root/.keras/datasets/aclImdb/train/pos/3319_9.txt',
 '/root/.keras/datasets/aclImdb/train/pos/4298_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/2363_7.txt',
 '/root/.keras/datasets/aclImdb/train/pos/12424_9.txt']

In [3]:
import os
import pandas as pd

def read(path):
  with open(path, 'r', encoding='utf-8') as f:
    content = f.read()
  return content

# get path
dn = os.path.dirname(dataset)
pattern = os.path.join(dn, 'aclImdb', 'train', 'pos', '*')
pos = glob.glob(pattern)
pattern = os.path.join(dn, 'aclImdb', 'train', 'neg', '*')
neg = glob.glob(pattern)
sentiments = [1]*len(pos) + [0]*len(neg)

contents = map(read, pos+neg)

pd.DataFrame(contents)

,0
0,The movie is great and I like the story. I pre...
1,This is of of Sammo's great early comedy films...
2,"As a long time Red Sox fan, I just had to go s..."
3,This movie awed me so much that I watch it at ...
4,This adaptation of M.R. James's short story 'A...
...,...
24995,It is unbelievable that a script as cliché and...
24996,A film like Amazing Grace and Chuck is a perfe...
24997,This is the biggest load of crap that I have s...
24998,"I awake suddenly, aware that I'm drooling onto..."


In [4]:
# map (similar as pd apply)
'-'.join(map(str, [1,2,3,4,5,6]))

'1-2-3-4-5-6'

In [5]:
def get_data(t):
  dn = os.path.dirname(dataset)
  pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
  pos = glob.glob(pattern)
  pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
  neg = glob.glob(pattern)
  sentiments = [1] * len(pos) + [0] * len(neg)
  contents = map(read, pos + neg)
  df = pd.DataFrame({
      "contents":contents,
      "sentiment":sentiments
  })
  return df

train_df = get_data('train')
test_df = get_data('test')
test_df

,contents,sentiment
0,"It kept me on the edge of my seat. True, the s...",1
1,Born in 1946 I was about eight years old when ...,1
2,In the 60's Cleveland television audiences cou...,1
3,I think it is a brilliant show with cool talki...,1
4,John Wayne's first starring role just blew me ...,1
...,...,...
24995,I am not a big fan of the Spielberg/Cruise ver...,0
24996,The director infuses this film with false dept...,0
24997,"As a flying and war movie buff, this ranks at ...",0
24998,I rented this movie under the impression that ...,0


In [6]:
# parameter
TOK = 3000
LEN = 512
EM = 128

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

tok = Tokenizer(num_words=TOK)
# fit_on_texts: fit_transform的fit
tok.fit_on_texts(train_df['contents'])
# seq: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(train_df['contents'])
x_test_seq = tok.texts_to_sequences(test_df['contents'])

pd.DataFrame(x_train_seq)
# One-hot encoding will cause massive loads on RAM.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,1,17,6,84,2,10,37,1,62.0,10.0,2781.0,11.0,17.0,71.0,82.0,17.0,138.0,1.0,2753.0,1189.0,17.0,2.0,705.0,17.0,10.0,40.0,373.0,1.0,879.0,10.0,37.0,3.0,632.0,2.0,79.0,3.0,280.0,62.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,6,4,4,84,399,209,105,11.0,215.0,3.0,2103.0,4.0,2537.0,1.0,2782.0,1.0,290.0,106.0,6.0,2186.0,16.0,1474.0,844.0,2.0,87.0,70.0,15.0,3.0,129.0,4.0,24.0,62.0,41.0,86.0,24.0,993.0,938.0,24.0,444.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14,3,193,55,764,334,10,40,66.0,5.0,137.0,64.0,1.0,17.0,9.0,13.0,84.0,134.0,47.0,67.0,112.0,27.0,192.0,409.0,926.0,36.0,1.0,764.0,808.0,47.0,68.0,84.0,660.0,4.0,46.0,4.0,58.0,511.0,764.0,1838.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,17,69,35,73,12,10,103,9.0,30.0,219.0,277.0,3.0,288.0,30.0,208.0,10.0,166.0,9.0,30.0,208.0,10.0,166.0,9.0,2.0,10.0,207.0,166.0,1.0,102.0,403.0,2.0,144.0,9.0,6.0,3.0,17.0,12.0,284.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1250,4,1976,1477,343,62,646,36.0,3.0,13.0,83.0,614.0,20.0,695.0,696.0,8.0,20.0,1.0,114.0,293.0,1305.0,2974.0,467.0,10.0,216.0,12.0,9.0,13.0,109.0,2444.0,171.0,20.0,2974.0,467.0,2.0,869.0,5.0,199.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,9,6,1295,12,3,226,14,1593,2.0,337.0,1753.0,97.0,94.0,98.0,265.0,57.0,1.0,389.0,28.0,1.0,411.0,8.0,11.0,17.0,163.0,303.0,37.0,3.0,309.0,1177.0,353.0,1511.0,295.0,1.0,75.0,2001.0,392.0,1643.0,1734.0,1298.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,3,19,37,477,1695,2,6,3,401.0,459.0,4.0,86.0,1.0,314.0,8.0,11.0,701.0,40.0,149.0,76.0,9.0,33.0,112.0,119.0,2.0,1992.0,5.0,1034.0,2.0,1034.0,80.0,989.0,7.0,7.0,11.0,19.0,2026.0,16.0,3.0,114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,6,1,1124,4,592,12,10,25.0,107.0,8.0,3.0,193.0,55.0,1.0,233.0,55.0,10.0,1796.0,3.0,17.0,35.0,73.0,13.0,1861.0,146.0,501.0,300.0,2.0,47.0,6.0,424.0,54.0,210.0,5.0,11.0,17.0,546.0,5.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,10,1081,1884,12,143,1644,1,1106,2.0,920.0,42.0,3.0,2265.0,2322.0,2660.0,135.0,13.0,10.0,2765.0,119.0,10.0,566.0,58.0,416.0,39.0,2503.0,29.0,4.0,58.0,97.0,9.0,27.0,1155.0,7.0,7.0,54.0,10.0,20.0,704.0,134.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# tok.word_index
tok.index_word[6]

'is'

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# padding: 截長or補短
# truncating: pre 12345 -> 45, post 12345 -> 12 
x_train_pad = pad_sequences(x_train_seq, LEN)
x_test_pad = pad_sequences(x_test_seq, LEN)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,17,6,84,2,10,37,1,62,10,2781,11,17,71,82,17,138,1,2753,1189,17,2,705,17,10,40,373,1,879,10,37,3,632,2,79,3,280,62,7,7
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,158,33,25,98,144,325,81,8,2576,1983,8,70,10,479,204,107,192,425,1211,14,605,1614,81,8,151,296,99,96,7,7,11,6,28,6,15,98,39,1474,844,334
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,76,5,810,1624,6,639,3,212,64,45,22,23,264,15,3,17,5,103,16,126,1301,39,2665,82,258,45,22,23,3,2353,334,2,57,50,258,45,22,23,764,334
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,157,2,254,82,15,125,39,430,1755,119,3,125,289,4,3,746,437,8,1,390,4,632,71,19,259,10,255,95,196,1768,8,65,203,93,11,19,6,3,212,64
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,71,1670,231,9,97,25,410,3,368,84,1250,14,9,6,9,29,761,3,114,2,3,224,50,5,267,1,1306,59,25,74,2373,7,7,10,199,9,690,43,4,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,18,89,448,1,94,65,903,14,70,1,862,274,77,126,122,45,22,116,39,141,24,2150,5,2220,43,57,50,10,654,26,97,199,69,145,104,631,4,58,110,142
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2770,34,1072,11,7,7,1,19,6,287,61,238,4,155,379,7,7,45,22,178,5,64,3,84,19,41,1,4,322,1227,16,881,302,7,7,440,22,23,54,1260
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,44,573,704,113,2,6,52,361,349,58,461,2,10,848,676,122,1,17,100,41,317,32,531,72,66,801,5,199,9,3,577,18,161,97,604,11,78,21,103,9
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,53,3,168,669,68,457,8,1,67,2,26,56,12,234,29,1,795,77,27,2356,54,28,77,147,86,370,9,6,2,26,56,67,398,1,1779,7,7,871,74,2808


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Flatten, Dropout

layers = [
      # 1 to 3000 (token) + 0 (padding)
      # param 3001 * 128
      Embedding(TOK+1, EM, mask_zero=True, input_length=LEN),
      Flatten(),
      Dense(128, activation='relu'),
      Dropout(0.25),
      Dense(2, activation='softmax')
]

model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
flatten (Flatten)            (None, 65536)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               8388736   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 8,773,122
Trainable params: 8,773,122
Non-trainable params: 0
_________________________________________________________________


In [11]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 5s 23ms/step - loss: 0.4914 - accuracy: 0.7495 - val_loss: 0.5751 - val_accuracy: 0.7424
Epoch 2/50
225/225 [==============================] - 5s 22ms/step - loss: 0.1855 - accuracy: 0.9306 - val_loss: 0.3410 - val_accuracy: 0.8644
Epoch 3/50
225/225 [==============================] - 5s 22ms/step - loss: 0.0486 - accuracy: 0.9872 - val_loss: 0.4151 - val_accuracy: 0.8640
Epoch 4/50
225/225 [==============================] - 5s 21ms/step - loss: 0.0084 - accuracy: 0.9988 - val_loss: 0.7239 - val_accuracy: 0.8208
Epoch 5/50
225/225 [==============================] - 5s 21ms/step - loss: 0.0020 - accuracy: 0.9998 - val_loss: 0.7476 - val_accuracy: 0.8372
Epoch 6/50
225/225 [==============================] - 5s 21ms/step - loss: 7.3071e-04 - accuracy: 1.0000 - val_loss: 0.8080 - val_accuracy: 0.8340
Epoch 7/50
225/225 [==============================] - 5s 21ms/step - loss: 3.7921e-04 - accuracy: 1.0000 - val_loss: 0.8612 - val_accuracy

In [12]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.3229 - accuracy: 0.8648


[0.3229168653488159, 0.864799976348877]

In [13]:
"""Smaller model to avoid overfitting"""
from tensorflow.keras.layers import GlobalAveragePooling1D

layers = [
  Embedding(TOK+1, EM, mask_zero=True, input_length=LEN),
  GlobalAveragePooling1D(),
  Dense(2, activation="softmax")  
]
model = Sequential(layers)
model.summary()
# This model is actually a linear model (no activation).

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 4s 18ms/step - loss: 0.5739 - accuracy: 0.7115 - val_loss: 0.4908 - val_accuracy: 0.7816
Epoch 2/50
225/225 [==============================] - 4s 18ms/step - loss: 0.3611 - accuracy: 0.8647 - val_loss: 0.3506 - val_accuracy: 0.8588
Epoch 3/50
225/225 [==============================] - 4s 18ms/step - loss: 0.2915 - accuracy: 0.8879 - val_loss: 0.3737 - val_accuracy: 0.8408
Epoch 4/50
225/225 [==============================] - 4s 18ms/step - loss: 0.2599 - accuracy: 0.8993 - val_loss: 0.3632 - val_accuracy: 0.8476
Epoch 5/50
225/225 [==============================] - 4s 18ms/step - loss: 0.2409 - accuracy: 0.9075 - val_loss: 0.3562 - val_accuracy: 0.8512
Epoch 6/50
225/225 [==============================] - 4s 18ms/step - loss: 0.2284 - accuracy: 0.9123 - val_loss: 0.3810 - val_accuracy: 0.8376
Epoch 7/50
225/225 [==============================] - 4s 18ms/step - loss: 0.2188 - accuracy: 0.9172 - val_loss: 0.3072 - val_accuracy: 0.8768

In [15]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.2904 - accuracy: 0.8820


[0.29037752747535706, 0.8820400238037109]

In [16]:
"""Transfer"""
layers = [
      Embedding(TOK+1, EM, mask_zero=True),
      GlobalAveragePooling1D()
]

w = model.layers[0].get_weights()
infer = Sequential(layers)
layers[0].set_weights(w)
infer.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         384128    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [17]:
infer.layers[0].get_weights()

[array([[ 0.01877122,  0.03873782,  0.00074854, ...,  0.02585527,
         -0.0065866 , -0.00646465],
        [ 0.03829893,  0.02506933,  0.03632063, ...,  0.03826848,
          0.01795508,  0.0297842 ],
        [ 0.09596263, -0.05440888,  0.07554679, ...,  0.03129658,
          0.00833689, -0.03639894],
        ...,
        [ 0.14380676, -0.13714564,  0.22556825, ...,  0.15595143,
          0.18871975, -0.14883824],
        [ 0.10300387, -0.17279036,  0.18344371, ...,  0.16140756,
          0.09643271, -0.11902371],
        [-0.03391658, -0.03665281, -0.01317866, ..., -0.02783183,
          0.01340682, -0.03651664]], dtype=float32)]

In [18]:
# 2500, 512
# [512 list, 512 list]
target = 'like'
# [1 list]
pre = infer.predict([[tok.word_index[target]]])
print(pre.shape)
print(pre[0])

(1, 128)
[ 0.0110106   0.02049715 -0.04145319 -0.02343697 -0.0018046   0.0106272
  0.01482004 -0.05147939 -0.0248849  -0.01767598  0.05076865 -0.00760286
  0.0359952   0.04986769  0.01305199  0.02681878  0.03877711  0.01103024
  0.05425582  0.05329145 -0.02682983 -0.06190083 -0.01563534  0.00368097
 -0.05282155 -0.01679191 -0.02330841  0.0354161   0.00476179  0.00881355
  0.01527095 -0.00973887 -0.01016273 -0.03073004 -0.0203518  -0.03193326
 -0.02422709 -0.02005108  0.05531799  0.0511295  -0.01819555 -0.02606977
 -0.02326293  0.0418388   0.03125253  0.02418105 -0.02223865 -0.00312008
 -0.01692042 -0.0474535   0.03213429 -0.0247573  -0.00287973  0.02196365
  0.01783885 -0.01400845  0.02511963  0.00625435 -0.03597102 -0.02981193
 -0.03035064 -0.02390301  0.04308075 -0.01036859 -0.03745437 -0.04617058
 -0.03566177  0.02298482  0.03957514 -0.02451827  0.03785603 -0.03440875
  0.00259391  0.03712018 -0.027287    0.0228284   0.03209416  0.01748884
 -0.04402437 -0.04567713  0.03384434 -0.009

In [19]:
f = open('vec.txt', 'w', encoding='utf-8')
f.write('{} {}\n'.format(TOK, EM))

for i in range(1, TOK+1):
  pre = infer.predict(([i]))[0]
  v = ' '.join(map(str, pre))
  f.write('{} {}\n'.format(tok.index_word[i], v))

f.close()

In [20]:
"""gensim"""
from gensim.models import KeyedVectors

w2v = KeyedVectors.load_word2vec_format('vec.txt')

w2v['good']

array([ 0.14513041, -0.14343497,  0.17053486, -0.12960589, -0.14743018,
       -0.12358911, -0.10958105,  0.15887275, -0.14647678, -0.09775795,
       -0.13067715,  0.13330267,  0.14194255, -0.07886307, -0.11237359,
       -0.1195811 , -0.09730654,  0.11776017, -0.1260057 , -0.17163523,
        0.12216855,  0.13905185,  0.15179767, -0.13088699,  0.12249795,
       -0.08323559,  0.16458346,  0.11177433,  0.15863109,  0.150033  ,
        0.1377535 , -0.15105288,  0.09511209, -0.1071227 , -0.08764897,
        0.07047936, -0.15512219, -0.13276178, -0.14630467, -0.12935513,
        0.15021445,  0.16949326, -0.13259944,  0.08071772,  0.13163222,
        0.16608569, -0.09156635, -0.13899119, -0.12288769,  0.18050152,
       -0.14190076,  0.11733617,  0.06155398,  0.08149419,  0.0673665 ,
       -0.09592164,  0.1514199 ,  0.07945747,  0.14105408, -0.08102929,
        0.15794796, -0.17408626, -0.11023658, -0.17411838,  0.12355185,
        0.12456926, -0.13544253,  0.15081282, -0.131845  , -0.16

In [21]:
w2v.wv.vocab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{'the': <gensim.models.keyedvectors.Vocab at 0x7f17c809c2e8>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7f17c809c208>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7f17c7feccf8>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7f17c4d77908>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7f17c57fa7f0>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7f17c57fa3c8>,
 'br': <gensim.models.keyedvectors.Vocab at 0x7f17c578b0b8>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7f17c5772588>,
 'it': <gensim.models.keyedvectors.Vocab at 0x7f17c57ce400>,
 'i': <gensim.models.keyedvectors.Vocab at 0x7f17c57ce198>,
 'this': <gensim.models.keyedvectors.Vocab at 0x7f17c57ce5c0>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7f17c57ce6a0>,
 'was': <gensim.models.keyedvectors.Vocab at 0x7f17c57ce748>,
 'as': <gensim.models.keyedvectors.Vocab at 0x7f17c57ce320>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7f17c57ce978>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7f17c57cea58>,
 'movie': <gensi

In [22]:
# 相似度比對 cos()
w2v.similarity('boyfriend', 'girlfriend')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9355887

In [23]:
w2v.most_similar('girlfriend')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('remotely', 0.9812001585960388),
 ('insult', 0.9811087250709534),
 ('awful', 0.980564296245575),
 ('disappointing', 0.9805143475532532),
 ('endless', 0.9803839921951294),
 ('contrived', 0.9803731441497803),
 ('uninteresting', 0.9803599119186401),
 ('waste', 0.9803577065467834),
 ('poorly', 0.9802834391593933),
 ('redeeming', 0.9799528121948242)]